In [1]:
from transformers import AutoTokenizer, RobertaForSequenceClassification
import torch
from tqdm import tqdm   
from underthesea import sent_tokenize, word_tokenize, sentiment
import pandas as pd
import pyhdfs
from pyspark.sql import * 
from pyspark.sql import functions as func 
from pyspark.sql import SparkSession

C:\Users\PHONG\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
spark = SparkSession.builder \
    .appName("Read CSV from HDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()


In [6]:

# Đọc dữ liệu từ file CSV và sử dụng dòng đầu tiên làm tên cột
df = spark.read.csv("hdfs://localhost:9000/big_data/coopmart_review_preprocess.csv", header=True, inferSchema=True)



In [7]:
# Chuyển đổi DataFrame của Spark thành DataFrame của pandas
df = df.toPandas()
df.head(5)

,comment,stars
0,nhiều hàng hoá để lựa chọn thường xuyên có chư...,5
1,dịch vụ tốt,5
2,hôm nay ngày lúc tầm giờ giờ tôi tính tiền ở q...,1
3,nhân viên tính tiền thái độ khó chịu ra mặt,1
4,thật tuyệt,5


In [ ]:
# Khởi tạo mô hình và tokenizer
checkpoint = "wonrax/phobert-base-vietnamese-sentiment"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=False)
model = RobertaForSequenceClassification.from_pretrained(checkpoint)

def get_sentiment(text, batch_size=64):
    # Định nghĩa mapping từ nhãn model sang nhãn mong muốn
    sentiment_mapping = {
        "POS": "Tích cực",
        "NEU": "Trung tính",
        "NEG": "Tiêu cực"
    }

    num_batches = len(text) // batch_size + (1 if len(text) % batch_size != 0 else 0)
    all_sentiments = []  # Lưu tất cả các cảm xúc dự đoán
    processed_count = 0  # Đếm số lượng đã xử lý

    for i in range(num_batches):
        batch_texts = text[i*batch_size:(i+1)*batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
        
        # Get predictions và tính softmax để lấy xác suất
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        for prediction in predictions:
            scores = prediction.tolist()
            highest_score_index = scores.index(max(scores))
            label_mapping = model.config.id2label
            sentiment_label = label_mapping[highest_score_index]
            
            # Map nhãn model sang nhãn mong muốn
            sentiment = sentiment_mapping[sentiment_label]
            all_sentiments.append(sentiment)
        
        processed_count += len(batch_texts)
        print(f"Đã xử lý: {processed_count}/{len(text)} văn bản")  # Hiển thị tiến trình
    
    # Tính cảm xúc phổ biến nhất từ tất cả các dự đoán
    sentiment_counter = Counter(all_sentiments)
    dominant_sentiment = sentiment_counter.most_common(1)[0][0]  # Chọn cảm xúc phổ biến nhất

    return dominant_sentiment

In [ ]:
df['sentiment']= df['comment'].apply(get_sentiment)

In [15]:
output_file = "coopmart_review_sentiment.csv"

# Lưu dataframe vào file csv
df.to_csv(output_file,encoding='utf-8-sig' ,index=False)